In [48]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
import xgboost
import lightgbm as lgm
import catboost

In [ ]:
train = pd.read_csv('train.csv' , parse_dates= ['date_created', 'user_date_created'] )
test = pd.read_csv('test.csv' ,parse_dates= ['date_created', 'user_date_created'] )

In [ ]:
train.shape , test.shape

In [ ]:
(train['date_created'].max() - train['date_created'].min() ).days , (test['date_created'].max() - test['date_created'].min() ).days

In [ ]:
train.shape , test.shape

In [ ]:
startTime_date_created = train['date_created'].min()
startTime_user_created = train['user_date_created'].min()
train['date_created'] = [ (date - startTime_date_created).days  for date in train['date_created']]
train['user_date_created'] = [ (date - startTime_date_created).days  for date in train['user_date_created']]
test['date_created'] = [ (date - startTime_user_created).days  for date in test['date_created']]
test['user_date_created'] = [ (date - startTime_user_created).days  for date in test['user_date_created']]
test.drop(columns = 'month_created')
train['is_fake'] = train['is_fake'].astype(int)

In [47]:
train

,comment_id,company_id,user_id,product_id,comment,rating,date_created,user_date_created,user_ip,user_agent,is_fake
0,1,2169053,91933,6.0,компания не выполнила заказ без обьяснения пр...,1,1709,-14,178.92.70.220,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,0
1,2,709306,91933,2.0,"Впервые встретился такой професиональный, отве...",5,2487,-14,94.179.59.162,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,0
2,3,35287,91933,NaN,"Несерйозная, некоректная менеджер інтернет маг...",1,2550,-14,37.73.137.188,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,0
3,4,1778322,91933,1.0,"Не рекомендую. Не отвечают, не выполняют заказы.",1,2685,-14,37.73.149.180,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,0
4,5,2821533,91933,NaN,"На письма не отвечают, заказы не выполняют. Не...",1,2695,-14,37.73.221.126,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,0
5,6,2770517,91933,NaN,я заказывала каталог с ценами по купальникам н...,1,1556,-14,95.132.127.210,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,0
6,7,2686753,91933,5.0,"Не понравилось, что пишут не верную информацию...",3,2438,-14,91.202.144.154,Mozilla/5.0 (Linux; Android 4.4.4; C6602 Build...,0
7,8,2686383,91951,4.0,"компания сработала хорошо, связались за пол дн...",5,2478,-14,95.132.4.59,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,0
8,9,2294801,91962,140.0,"Все понравилось. Качественный товар, заказ был...",5,2296,-14,212.90.61.2,Mozilla/5.0 (Windows NT 6.1; rv:46.0) Gecko/20...,0
9,10,1879360,91989,NaN,Имел неосторожность купить в 2013 году после в...,1,1379,-14,91.241.146.34,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,0


# Val

In [14]:
train.shape[0]/ test.shape[0]

5.083316474876933

In [ ]:
from sklearn.model_selection import StratifiedKFold , cross_val_score , GridSearchCV
X = train.drop(columns = 'is_fake')
y = train['is_fake'].astype(int)
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42)

In [36]:
test

,comment,comment_id,company_id,date_created,month_created,product_id,rating,user_agent,user_date_created,user_id,user_ip
0,"спасибо все отлично, качество как на картинке,...",1655822,2214161,2018-03-19 21:36:07,2018-03,1301549.0,5,NaN,2017-11-01 00:00:46,12969242,NaN
1,Заказывала шоколад Риттер Спорт 10 шт. К сожал...,1655823,2333068,2017-12-10 13:46:49,2017-12,699700.0,5,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2017-11-01 00:02:19,12969248,91.90.13.210
2,"Быстро отзвонились, охотно отвечали на все воп...",1655824,752692,2017-11-01 00:07:38,2017-11,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2017-11-01 00:02:44,12969251,93.115.95.204
3,Очень оперативный и внимательный к покупателю ...,1655825,334751,2017-11-20 23:40:01,2017-11,1279247.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
4,Все прошло очень оперативно! Заказ привезли в ...,1655826,2263650,2017-11-20 23:39:51,2017-11,1225545.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
5,Отличная компания. Очень оперативно и качестве...,1655827,1958752,2017-11-20 23:40:16,2017-11,1299459.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
6,Все четко. Заказал наложенным платежом. Пришел...,1655828,2840325,2017-11-20 23:39:45,2017-11,1301564.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
7,"Компетентность, оперативность, пунктуальность....",1655829,841233,2017-11-20 23:39:36,2017-11,1222815.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
8,Такой быстрой доставки еще не встречал. Все пр...,1655830,1981477,2017-11-01 00:07:01,2017-11,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2017-11-01 00:02:44,12969251,93.115.95.204
9,"Заказ выполнен в срок, адекватная цена, без пр...",1655831,333335,2017-11-20 23:40:39,2017-11,1301991.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28


In [41]:
train

,comment_id,company_id,user_id,product_id,comment,rating,date_created,user_date_created,user_ip,user_agent,is_fake
0,1,2169053,91933,6.0,компания не выполнила заказ без обьяснения пр...,1,1709,2010-02-03 21:03:22,178.92.70.220,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,0
1,2,709306,91933,2.0,"Впервые встретился такой професиональный, отве...",5,2487,2010-02-03 21:03:22,94.179.59.162,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,0
2,3,35287,91933,NaN,"Несерйозная, некоректная менеджер інтернет маг...",1,2550,2010-02-03 21:03:22,37.73.137.188,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,0
3,4,1778322,91933,1.0,"Не рекомендую. Не отвечают, не выполняют заказы.",1,2685,2010-02-03 21:03:22,37.73.149.180,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,0
4,5,2821533,91933,NaN,"На письма не отвечают, заказы не выполняют. Не...",1,2695,2010-02-03 21:03:22,37.73.221.126,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,0
5,6,2770517,91933,NaN,я заказывала каталог с ценами по купальникам н...,1,1556,2010-02-03 21:03:22,95.132.127.210,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,0
6,7,2686753,91933,5.0,"Не понравилось, что пишут не верную информацию...",3,2438,2010-02-03 21:03:22,91.202.144.154,Mozilla/5.0 (Linux; Android 4.4.4; C6602 Build...,0
7,8,2686383,91951,4.0,"компания сработала хорошо, связались за пол дн...",5,2478,2010-02-03 22:17:55,95.132.4.59,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,0
8,9,2294801,91962,140.0,"Все понравилось. Качественный товар, заказ был...",5,2296,2010-02-03 23:03:27,212.90.61.2,Mozilla/5.0 (Windows NT 6.1; rv:46.0) Gecko/20...,0
9,10,1879360,91989,NaN,Имел неосторожность купить в 2013 году после в...,1,1379,2010-02-04 03:08:56,91.241.146.34,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,0


In [5]:
pp.ProfileReport(train)

Number of variables,11
Number of observations,1281499
Total Missing (%),3.2%
Total size in memory,99.0 MiB
Average record size in memory,81.0 B
Numeric,4
Categorical,4
Boolean,1
Date,1
Text (Unique),0
Rejected,1


In [6]:
pp.ProfileReport(test)

Number of variables,11
Number of observations,252099
Total Missing (%),10.0%
Total size in memory,21.2 MiB
Average record size in memory,88.0 B
Numeric,4
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [7]:
train['date_created']

0         2014-10-23 20:26:44
1         2016-12-10 08:38:27
2         2017-02-11 15:55:14
3         2017-06-26 18:00:45
4         2017-07-06 16:57:40
5         2014-05-23 21:06:03
6         2016-10-22 12:35:24
7         2016-12-01 13:01:50
8         2016-06-01 21:55:43
9         2013-11-28 16:26:08
10        2014-07-01 14:17:03
11        2014-04-04 08:40:55
12        2014-08-21 14:08:45
13        2015-11-30 09:39:38
14        2012-12-26 23:29:36
15        2013-12-15 11:44:14
16        2015-08-05 10:06:28
17        2016-10-30 11:52:38
18        2013-10-24 18:41:23
19        2016-07-14 11:13:01
20        2017-05-12 20:05:40
21        2016-10-27 19:26:29
22        2013-06-08 11:42:08
23        2016-05-03 23:16:08
24        2016-08-24 02:31:51
25        2017-08-26 13:58:46
26        2014-05-03 12:43:54
27        2013-10-26 19:03:21
28        2014-03-09 14:24:18
29        2017-03-23 13:20:26
                  ...        
1281469   2017-10-31 12:02:19
1281470   2017-10-31 14:46:48
1281471   